<a href="https://colab.research.google.com/github/shahkarKhan24/Natural-Language-Inference-NLI/blob/main/Data_augmentation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install --upgrade datasets

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   -- ------------------------------------ 41.0/547.8 kB 991.0 kB/s eta 0:00:01
   -------------------------------- ------- 450.6/547.8 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 547.8/547.8 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/402.6 kB ? eta -:--:--
   --------------------------------------  399.4/402.6 kB 25.9 MB/s eta 0:00:01
   ---------------------------------------- 402.6/402.6 kB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: huggingface-h

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires spacy<3.4,>=2.1.0, but you have spacy 3.7.5 which is incompatible.
cached-path 1.1.6 requires huggingface-hub<0.11.0,>=0.8.1, but you have huggingface-hub 0.23.4 which is incompatible.
openai 1.12.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.12.0 requires pydantic<3,>=1.9.0, but you have pydantic 1.8.2 which is incompatible.
openai 1.12.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
sentence-transformers 2.3.1 requires transformers<5.0.0,>=4.32.0, but you have transformers 4.20.1 which is incompatible.


# Generating Adversarial Dataset:
In this task also have to deal with Adversarial-NLI, so modifying the data in a way that the model is more robust for NLI task, so we are asked to generate an adversarial data using original dataset. In order to do that, I used several techniques all combined together to generate my own adversarial dataset comprising of 15000 samples. The focus was on augmenting the hypothesis of each sample, as this is less complex than modifying the premise.
The data augmentation techniques utilized include antonym substitution, synonym substitution, and hypernym substitution, facilitated by the Word Sense Disambiguation (WSD) annotations provided in the original dataset. For antonym substitution, the label was also switched (e.g., contradiction to entailment and vice versa). Additionally, I used cross-lingual augmentation technique, involving back-translation, which proved to be the most sophisticated method in preserving the original sentence's semantics, but at the cost of increased processing time. The other technique that I used were very generic because using semantic alone may or may not be beneficial, so I used some non-semantic augmentation but in a very small percentage. One such technique was word-swap, where subjects or objects within a sentence were exchanged, resulting in slight disorganization without losing semantic meaning. And finally, I used Noise insertion technique which simply find a random words or letter in each sentence and add it again in a sentence. This technique can help the model to be more robust to any typos that may be present in a data.
We also notice that there were many cases where some technique might not work for a sentence, in that case I simply used again the synonym substitution technique which was way more reliable if any technique fails. Each technique was assigned a specific percentage to determine its application across the subset of 15,000 samples, as detailed in the accompanying table. The techniques assigned a lower percentage were chosen as such due to their higher likelihood of causing a loss of semantic meaning in the sentence. The augmenting took around 3 hours to process for 15000 samples, which were than combine and shuffled across the original training dataset  


In [ ]:
from nltk.corpus import wordnet as wn
import random
import nltk
import pandas as pd
from datasets import load_dataset, Dataset
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from googletrans import Translator
from tqdm import tqdm
import time
import spacy


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shahk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shahk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
nlp = spacy.load("en_core_web_sm")

this cell was run to clear the cache to redownlad the dataset

In [ ]:
# import os
# import shutil
# # Path to the cache directory
# cache_dir = "C:/Users/shahk/.cache/huggingface/datasets/tommasobonomo___parquet/tommasobonomo--sem_augmented_fever_nli-7de7767a134493f7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/"
# # Delete the cache directory if it exists
# if os.path.exists(cache_dir):
#     shutil.rmtree(cache_dir)
#     # Set an environment variable to disable caching
# os.environ['HF_DATASETS_CACHE'] = '/path/to/non_existent_directory'

In [ ]:
# Load the dataset
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", download_mode="force_redownload")

# View the dataset
print(dataset)

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})


In [ ]:
# Select the first 15000 samples using the indices
# Get the indices of the first 15000 samples
indices = list(range(15000))
subset_dataset = dataset['train'].select(indices)

In [ ]:

print(dataset['train'].features)
print(subset_dataset.features)

{'id': Value(dtype='string', id=None), 'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None), 'wsd': {'premise': [{'index': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'pos': Value(dtype='string', id=None), 'lemma': Value(dtype='string', id=None), 'bnSynsetId': Value(dtype='string', id=None), 'wnSynsetOffset': Value(dtype='string', id=None), 'nltkSynset': Value(dtype='string', id=None)}], 'hypothesis': [{'index': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'pos': Value(dtype='string', id=None), 'lemma': Value(dtype='string', id=None), 'bnSynsetId': Value(dtype='string', id=None), 'wnSynsetOffset': Value(dtype='string', id=None), 'nltkSynset': Value(dtype='string', id=None)}]}, 'srl': {'premise': {'tokens': [{'index': Value(dtype='int64', id=None), 'rawText': Value(dtype='string', id=None)}], 'annotations': [{'tokenIndex': Value(dtype='int64', id=None), 'verba

<h3>Sysnonyms, Hypernyms and Antonyms Substitution Using WSD Annotation

In [ ]:
# Function to map Universal POS tags to WordNet POS tags
def map_pos_universal_to_wordnet(universal_pos):
    pos_map = {
        'ADJ': wn.ADJ,
        'ADP': wn.ADV,   # Mapping ADP to ADV (adverb in WordNet)
        'ADV': wn.ADV,
        'AUX': wn.VERB,  # Mapping AUX to VERB in WordNet
        'CCONJ': None,   # WordNet does not directly support coordinating conjunctions
        'DET': None,     # WordNet does not directly support determiners
        'NOUN': wn.NOUN,
        'NUM': None,     # No direct support for numbers in WordNet
        'PART': wn.ADV,  # Mapping PART to ADV (adverb in WordNet)
        'PRON': wn.NOUN, # Mapping PRON to NOUN in WordNet
        'PROPN': wn.NOUN, # Mapping PROPN to NOUN in WordNet
        'PUNCT': None,   # WordNet does not directly support punctuation
        'SYM': wn.NOUN,  # Mapping SYM to NOUN in WordNet
        'VERB': wn.VERB
    }
    return pos_map.get(universal_pos)
# Function to get synonyms using WordNet and POS tag
def get_synonyms(word, universal_pos):
    wn_pos = map_pos_universal_to_wordnet(universal_pos)

    if wn_pos:
        synsets = wn.synsets(word, pos=wn_pos)
        synonyms = set()
        for synset in synsets:
            for lemma in synset.lemmas():
                synonym = lemma.name().replace('_', ' ')
                if synonym.lower() != word.lower():
                    synonyms.add(synonym)
        return list(synonyms)
    else:
        return []

# Function to get hypernyms using WordNet and Universal POS tag
def get_hypernyms(word, universal_pos):
    wn_pos = map_pos_universal_to_wordnet(universal_pos)

    if wn_pos:
        synsets = wn.synsets(word, pos=wn_pos)
        hypernyms = set()
        for synset in synsets:
            for hypernym in synset.hypernyms():
                for lemma in hypernym.lemmas():
                    hypernym_word = lemma.name().replace('_', ' ')
                    if hypernym_word.lower() != word.lower():
                        hypernyms.add(hypernym_word)
        return list(hypernyms)
    else:
        return []


# Function to get antonyms using WordNet and POS tag
def get_antonyms(word, universal_pos):
    wn_pos = map_pos_universal_to_wordnet(universal_pos)

    if wn_pos:
        antonyms = set()
        for synset in wn.synsets(word, pos=wn_pos):
            for lemma in synset.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name().replace('_', ' '))
        return list(antonyms)
    else:
        return []


In [ ]:
# Function to apply synonym substitution on hypothesis

def apply_synonym_substitution(augmented_hyp, wsd_annotations):
    techniques = ["synonym",  "hypernym"]
    technique = random.choice(techniques)
    count=0
    to_replace = random.randint(1,len(wsd_annotations))-1 # random index for first wsd annoation of the hypothesis, mean random word
    check=wsd_annotations[to_replace]['wnSynsetOffset']
    tag= wsd_annotations[to_replace]['pos']

    if check == 'O':
        count=0
        while check in ['O']:
            if count==len(wsd_annotations):
                break
            else:
                to_replace = random.randint(0, len(wsd_annotations) - 1)
                check = wsd_annotations[to_replace]['wnSynsetOffset']
                tag = wsd_annotations[to_replace]['pos']
                count+=1
    if count==len(wsd_annotations):
        return augmented_hyp

    # # Check if tag is in the list of tags to avoid
    # if tag in ['CCONJ', 'DET', 'NUM', 'PART', 'PUNCT', 'SYM','PROPN']:
    #     count=0
    # # If the tag matches one of the specified ones, find a new to_replace
    #     while tag in ['CCONJ', 'DET', 'NUM', 'PART', 'PUNCT', 'SYM','PROPN']:
    #         if count==len(wsd_annotations):
    #             break
    #         else:
    #             to_replace = random.randint(0, len(wsd_annotations) - 1)
    #             tag = wsd_annotations[to_replace]['pos']
    #             count+=1
    # if count==len(wsd_annotations):
    #     return augmented_hyp


    word=wsd_annotations[to_replace]['text']
    if technique == "synonym":
        synonyms=get_synonyms(word, tag)
    elif technique == "hypernym":
        synonyms=get_hypernyms(word, tag)

    if synonyms:
        synonym = random.choice(synonyms)
        count2=0
        while synonym==word and count2!=len(synonyms):
            synonyms.remove(word)
            synonym = random.choice(synonyms)
            count2+=1

        augmented_hyp = augmented_hyp.replace(word, synonym)

    return augmented_hyp

In [ ]:
# Function to apply antonym substitution on hypothesis
def apply_antonym_substitution(augmented_hyp, wsd_annotations, augmented_labels):
    count = 0
    to_replace = random.randint(0, len(wsd_annotations) - 1)  # Ensure the random index is within range
    tag = wsd_annotations[to_replace]['pos']
    check=wsd_annotations[to_replace]['wnSynsetOffset']

    if check == 'O':
        count=0
        while check in ['O']:
            if count==len(wsd_annotations):
                break
            else:
                to_replace = random.randint(0, len(wsd_annotations) - 1)
                check = wsd_annotations[to_replace]['wnSynsetOffset']
                tag = wsd_annotations[to_replace]['pos']
                count+=1

    if count==len(wsd_annotations):
        return augmented_hyp, augmented_labels # if sentence is empty send the same sentence back

    # # Check if tag is in the list of tags to avoid
    # if tag in ['CCONJ', 'DET', 'NUM', 'PART', 'PUNCT', 'SYM','ADV','ADP']:
    #     count = 0
    #     while tag in ['CCONJ', 'DET', 'NUM', 'PART', 'PUNCT', 'SYM', 'ADV', 'ADP']:
    #         if count == len(wsd_annotations):
    #             break
    #         else:
    #             to_replace = random.randint(0, len(wsd_annotations) - 1)
    #             tag = wsd_annotations[to_replace]['pos']
    #             count += 1

    # if count == len(wsd_annotations):

    #     return augmented_hyp, augmented_labels

    word = wsd_annotations[to_replace]['text']
    antonyms = get_antonyms(word, tag)

    if antonyms != []:
        antonym = random.choice(antonyms)
        count2 = 0
        while antonym == word and count2 != len(antonyms):
            antonyms.remove(word)
            antonym = random.choice(antonyms)
            count2 += 1

        augmented_hyp = augmented_hyp.replace(word, antonym)  # Updated this line to replace in the string

        if augmented_labels == 'ENTAILMENT':
            augmented_labels = 'CONTRADICTION'
        elif augmented_labels == 'CONTRADICTION':
            augmented_labels = 'ENTAILMENT'
        else:
            augmented_labels ='NEUTRAL'

    else:
        synonyms=get_synonyms(word, tag)
        if synonyms:
            synonym = random.choice(synonyms)
            count2=0
            while synonym==word and count2!=len(synonyms):
                synonyms.remove(word)
                synonym = random.choice(synonyms)
                count2+=1

            # synonym=synonyms[0]
            augmented_hyp = augmented_hyp.replace(word, synonym)


    return augmented_hyp, augmented_labels


<h3>Back Translation<h3>

I used cross-lingual augmentation technique, involving back-translation, which proved to be the most sophisticated method in preserving the original sentence's semantics, but at the cost of increased processing time.
Its a bit time consuming it takes around 23 minute to back-translate 1000 sentence

In [ ]:
# Function to perform back translation with retry logic
def Cross_lingual_augmentation(text, src_lang='en', mid_lang='it', retries=3):
    translator = Translator()
    for attempt in range(retries):
        try:
            # Translate text to intermediate language
            translated = translator.translate(text, src=src_lang, dest=mid_lang).text
            # Translate back to the original language
            back_translated = translator.translate(translated, src=mid_lang, dest=src_lang).text
            return back_translated
        except Exception as e:
            print(f"Error: {e}, retrying back translation... ({attempt + 1}/{retries}) for text: {text}")
            time.sleep(1)  # Wait before retrying
    return text  # Return original text if all retries fail

<h3>Noise_Insertion



I used Noise insertion technique which simply find a random words or letter in each sentence and add it again in a sentence. This technique can help the model to be more robust to any typos that may be present in a data.

In [ ]:
def random_insertion(sentence):
    n=1
    words = sentence.split()
    for _ in range(n):
        new_word = random.choice(words)
        position = random.randint(0, len(words))
        words.insert(position, new_word)
    return ' '.join(words)


def noise_injection(sentence):
    noise_level=0.07
    words = list(sentence)
    num_noises = int(len(words) * noise_level)
    for _ in range(num_noises):
        pos = random.randint(0, len(words) - 1)
        words[pos] = random.choice('abcdefghijklmnopqrstuvwxyz')
    return ''.join(words)


# Function to apply back translation on hypothesis
def Noise_augmentation(hypothesis):
    techniques = ["word",  "letter"]
    technique = random.choice(techniques)
    if technique == "word":
        augmented_sentence=random_insertion(hypothesis)
    elif technique == "letter":
        augmented_sentence=noise_injection(hypothesis)

    return augmented_sentence

<h3>Words Swapping

The technique word-swap, where subjects or objects within a sentence were exchanged, resulting in slight disorganization without losing semantic meaning.

In [ ]:
# Function to get synonyms from WordNet
def get_synonyms_wo_pos(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to swap two words in a sentence
def swap_augmentaion(sentence):
    doc = nlp(sentence)

    # Identify words to swap (e.g., subject and object)
    words_to_swap = []
    for token in doc:
        if token.dep_ in {"nsubj", "dobj"}:  #
            words_to_swap.append((token.text, token.i))
        # if len(words_to_swap) == 2:
        #     break
    if len(words_to_swap) >= 2:
        random_words_to_swap = random.sample(words_to_swap, 2)
    # If less than two words are found, no swapping is possible
    if len(words_to_swap) == 0:
        return sentence

    if len(words_to_swap) < 2:
        # synonyms=get_synonyms_wo_pos(words_to_swap[0][0])

        # if synonyms:
        #     synonym = random.choice(synonyms)
        #     count2=0
        #     while synonym==words_to_swap[0][0]and count2!=len(synonyms):
        #         # synonyms.remove(words_to_swap[0][0])
        #         synonym = random.choice(synonyms)
        #         count2+=1

        #     # synonym=synonyms[0]
        #     sentence = sentence.replace(words_to_swap[0][0], synonym)
        return sentence

    # Swap the positions of the identified words
    word1, index1 = random_words_to_swap[0]
    word2, index2 = random_words_to_swap[1]

    # Convert doc to list of tokens
    tokens = [token.text for token in doc]

    # Swap the words in the token list
    tokens[index1], tokens[index2] = tokens[index2], tokens[index1]

    # Join the tokens back into a sentence
    swapped_sentence = " ".join(tokens)
    return swapped_sentence

Each technique was assigned a specific percentage to determine its application across the subset of 15,000 samples, The techniques assigned a lower percentage were chosen as such due to their higher likelihood of causing a loss of semantic meaning in the sentence. The augmenting took around 3 hours to process for 15000 samples, which were than combine and shuffled across the original training dataset  

In [ ]:
augmentation_percentages = {
    "synonym": 0.5,
    "antonym": 0.1,
    "swap": 0.1,
    "noise": 0.1,
    "back_translation": 0.2,
}
def calculate_samples_per_technique(dataset_size, percentages):
    num_samples = {}
    for technique, percentage in percentages.items():
        num_samples[technique] = int(dataset_size * percentage)
    return num_samples

# Example usage
dataset_size = len(subset_dataset)
num_samples = calculate_samples_per_technique(dataset_size, augmentation_percentages)
print(num_samples)

{'synonym': 7500, 'antonym': 1500, 'swap': 1500, 'noise': 1500, 'back_translation': 3000}


In [ ]:
subset_dataset

Dataset({
    features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
    num_rows: 15000
})

In [ ]:
def augment_samples(example):
    used=[]# to see which technique was used during each sample to augment
    dataset_size = len(example)
    num_samples = calculate_samples_per_technique(dataset_size, augmentation_percentages)
    # num_samples
    aug_count={'synonym': 0, 'antonym': 0, 'swap': 0, 'noise': 0, 'back_translation': 0}
    techniques = ["synonym","antonym","swap","noise","back_translation"]

    augmented_hyp = example['hypothesis'][:]
    augmented_labels= example['label'][:]
    wsd = example['wsd']
    wsd_annotations = []

    for i in range(len(wsd)):
        extracted_hyp = wsd[i]['hypothesis']
        wsd_annotations.append(extracted_hyp)

    for i in tqdm(range(len(example['hypothesis'])), desc="Augmenting"):
        repeat=True
        attempt=len(example['hypothesis'][i])
        while repeat and attempt !=0:
            technique=random.choice(techniques)
            if technique == 'synonym' and aug_count['synonym']< num_samples["synonym"]:
                new_hyp= apply_synonym_substitution(example['hypothesis'][i], wsd_annotations[i])
                attempt-=1
                if new_hyp!=augmented_hyp[i]:
                    used.append(technique)
                    augmented_hyp[i]=new_hyp
                    repeat=False
                    aug_count['synonym']+=1

            elif technique == 'antonym' and aug_count['antonym'] < num_samples["antonym"]:
                new_hyp, new_label= apply_antonym_substitution(example['hypothesis'][i], wsd_annotations[i],augmented_labels[i])
                attempt-=1
                if new_hyp!=augmented_hyp[i]:
                    used.append(technique)
                    augmented_hyp[i],augmented_labels[i]=new_hyp, new_label
                    repeat=False
                    aug_count['antonym']+=1

            elif technique == 'swap' and aug_count['swap'] < num_samples["swap"]:
                new_hyp = swap_augmentaion(example['hypothesis'][i])
                attempt-=1
                if new_hyp!=augmented_hyp[i]:
                    used.append(technique)
                    augmented_hyp[i]=new_hyp
                    repeat=False
                    aug_count['swap']+=1
                if attempt==0 and new_hyp==augmented_hyp[i]:# if we cant find words to swap after many attempts than apply synonym substitution
                    new_hyp= apply_synonym_substitution(example['hypothesis'][i], wsd_annotations[i])
                    augmented_hyp[i]=new_hyp
                    aug_count['swap']+=1


            elif technique == 'noise' and aug_count['noise'] < num_samples["noise"]:
                new_hyp = Noise_augmentation(example['hypothesis'][i])
                attempt-=1
                if new_hyp!=augmented_hyp[i]:
                    used.append(technique)
                    augmented_hyp[i]=new_hyp
                    repeat=False
                    aug_count['noise']+=1


            elif technique == 'back_translation' and aug_count['back_translation'] < num_samples["back_translation"]:
                new_hyp = Cross_lingual_augmentation(example['hypothesis'][i])
                # attempt==0
                used.append(technique)
                if new_hyp==augmented_hyp[i]:# if the traslated version is same as orignal than apply synonym substitution
                   new_hyp= apply_synonym_substitution(example['hypothesis'][i], wsd_annotations[i])

                augmented_hyp[i]=new_hyp
                repeat=False
                aug_count['back_translation']+=1


            if attempt==0:
                used.append('Same as original')







    return augmented_hyp,augmented_labels, used




In [ ]:
a_h, a_l, a_t=augment_samples(subset_dataset)

Augmenting: 100%|██████████| 15000/15000 [1:17:09<00:00,  3.24it/s] 


In [ ]:
# Ensure the length of the new hypothesis list matches the number of rows in the dataset
assert len(a_h) == subset_dataset.num_rows

In [ ]:
# Replace the hypothesis column
def replace_hypothesis(example, idx):
    example['hypothesis'] = a_h[idx]
    example['label'] = a_l[idx]
    return example

In [ ]:
# aug_dataset = subset_dataset.map(replace_hypothesis, with_indices=True)
aug_dataset = subset_dataset.map(replace_hypothesis, with_indices=True)


In [ ]:
aug_dataset.save_to_disk('adverserial_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
Adverserial_Dataset = load_from_disk('adverserial_dataset')

In [ ]:
print(dataset['train'].features)
print(Adverserial_Dataset.features)

{'id': Value(dtype='string', id=None), 'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None), 'wsd': {'premise': [{'index': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'pos': Value(dtype='string', id=None), 'lemma': Value(dtype='string', id=None), 'bnSynsetId': Value(dtype='string', id=None), 'wnSynsetOffset': Value(dtype='string', id=None), 'nltkSynset': Value(dtype='string', id=None)}], 'hypothesis': [{'index': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'pos': Value(dtype='string', id=None), 'lemma': Value(dtype='string', id=None), 'bnSynsetId': Value(dtype='string', id=None), 'wnSynsetOffset': Value(dtype='string', id=None), 'nltkSynset': Value(dtype='string', id=None)}]}, 'srl': {'premise': {'tokens': [{'index': Value(dtype='int64', id=None), 'rawText': Value(dtype='string', id=None)}], 'annotations': [{'tokenIndex': Value(dtype='int64', id=None), 'verba

Combining and shuffling with original training dataset

In [ ]:
from datasets import Dataset, DatasetDict, concatenate_datasets

# Assuming you have already loaded your datasets, e.g., first_dataset and second_dataset

# Combine the train set of the first dataset with the second dataset
combined_train_dataset = concatenate_datasets([dataset['train'], Adverserial_Dataset])

In [ ]:
# Shuffle the combined dataset
shuffled_combined_train_dataset = combined_train_dataset.shuffle(seed=42)

# Replace the train set in the first dataset with the shuffled combined dataset
combined_dataset_dict = DatasetDict({
    'train': shuffled_combined_train_dataset,
    'validation': dataset['validation'],
    'test': dataset['test']
})

In [ ]:
# Print to verify
print(combined_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 66086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})


In [ ]:
ran= random.randint(0,499)
print(f"auggmented hypothesis: {aug_dataset[ran]['hypothesis']}, Label: {aug_dataset[ran]['label']}, Technique used: {a_t[ran]}, length:{len(aug_dataset[ran]['hypothesis'])} ")
print(f"original hypothesis:   {subset_dataset[ran]['hypothesis']}, Label: {subset_dataset[ran]['label']}, length:{len(subset_dataset[ran]['hypothesis'])}")

auggmented hypothesis: Michael Fassbender is an worker., Label: ENTAILMENT, Technique used: synonym, length:32 
original hypothesis:   Michael Fassbender is an actor., Label: ENTAILMENT, length:31
